# Homework 1. MNIST 데이터셋을 활용해 다중 분류 수행하기
### 과목코드: [입력하시오] (2150034001/2150085101)

#### 문제. 아래의 `코드`를 완성하고, 그에 대한 `분석`을 작성하시오.

1. (코드) MNIST 데이터셋 받기
   - (분석) 데이터셋에 대해 분석하기

2. (코드) 데이터셋을 train / validation / test set 으로 분할하기
   - (분석) 데이터셋을 분할한 방식에 대해 설명하고, 해당 분할 방식을 수행한 이유에 대해 설명하기

3. (코드) 특성 스케일링을 이용한 데이터셋 전처리하기
   - (분석) 스케일링을 수행한 방식에 대해 설명하고, 해당 스케일링을 수행한 이유에 대해 설명하기

4. (코드) 모델 선택하기
   - (분석) 머신러닝 모델의 후보군들에 대해 설명하고, 후보군들 중 특정 모델이 대표 모델로 선택된 이유에 대해 설명하기

1. 로지스틱 회귀 (Logistic Regression)
로지스틱 회귀는 Binary Classfication(이진 분류)를 위해 사용되지만 Softmax Regression을 사용하면 다중 클래스 분류에도 사용이 가능하다.
하지만 데이터의 선형 관계를 이용해 분석하는 로지스틱 회귀는 데이터가 비선형적인(4와 9의 유사성 등) MNIST에는 사용하기 부적합하다고 판단했다.

2. kNN, 최근접이웃 (k-Nearest Neighbors) 
새로운 데이터를 가장 가까운 k개의 이웃의 class를 참조하여 분류하는 kNN 알고리즘을 이용해 분류할 수 있다.
그러나 kNN은 새로운 데이터 포인트를 분류할 때 기존의 모든 데이터와의 거리를 계산해야 하므로, 데이터셋이 클수록 계산 비용이 급격히 증가하는 단점이 있다.
또한 단순히 이웃의 class를 기반으로 분류를 결정하기 때문에, 분류 결과에 대한 직관적인 설명이 어렵다.
Scikit learn 으로 구현한 사례가 존재한다. (https://teddylee777.github.io/scikit-learn/scikit-learn-knn/)

3. CNN, 합성곱 신경망 (Convolutional Neural Networks)
CNN은 이미지 분류 작업에서 매우 효과적인 알고리즘임을 확인할 수 있었다. 특히 데이터가 비선형적이거나 복잡한 패턴을 포함하고 있을 때 높은 성능을 보여주므로
이번 Homework에는 MNIST를 분류하는데 CNN 모델을 이용하였다.

//또한 수업시간에 다루었던 확률적 경사 하강 모델(SGD) , 랜덤포레스트 모델, 결정트리 모델을 이용해 MNIST를 분류해보고 CNN 모델과 비교해보았다.

5. (코드) 성능 평가하기
   - (분석) 최종적으로 학습된 모델을 이용해 test set 에 대한 성능을 정리하고, 이를 분석하기

6. (코드) 에러 분석하기
   - (분석) 최종적으로 학습된 모델의 에러를 분석하기
   - (분석) `5. 성능 평가하기` 의 분석과 연관지어 에러를 분석하고 해결책을 제안하기

참고자료